In [6]:
import os
import json
import wave
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from vosk import Model, KaldiRecognizer
import soundfile as sf
import librosa
from transformers import BertTokenizer, BertModel
import noisereduce as nr
from sklearn.metrics import classification_report

# Константы
DATA_DIR = '../data/train/'
DATA_DIR_FILES = [
    # 'hr_bot_clear',
    # 'hr_bot_noise',
    'hr_bot_synt'
]
ANNOTATION_DIR = '../data/train/annotation/'
ANNOTATION_FILES = [
    # 'hr_bot_clear.json',
    # 'hr_bot_noise.json',
    'hr_bot_synt.json'
]

VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_VAL_FILE = os.path.join(VAL_DIR, 'luga.json')

# Инициализация модели Vosk для распознавания речи
model = Model(MODEL_PATH)
print("Модель Vosk загружена успешно.")

# Классы команд
_labels = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}

# Класс для датасета
class AudioTextDataset(Dataset):
    def __init__(self, audio_files, texts, labels):
        self.audio_files = audio_files
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.audio_files[idx], self.texts[idx], self.labels[idx]

# Функция для удаления шума
def reduce_noise(audio_file):
    audio, sr = librosa.load(audio_file, sr=16000)
    # Удаление шума
    clean_audio = nr.reduce_noise(y=audio, sr=sr)
    # Сохранение очищенного аудио во временный файл
    cleaned_file = "temp_cleaned.wav"
    sf.write(cleaned_file, clean_audio, sr)
    return cleaned_file

# Функция для распознавания речи
def transcribe_audio(audio_file):
    wf = wave.open(audio_file, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    result_text = ""

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            result_text += result.get("text", "")

    final_result = json.loads(rec.FinalResult())
    result_text += final_result.get("text", "")
    return result_text.strip()

# Класс для текстового классификатора
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
# Аугментация аудио
def augment_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    noise = np.random.randn(len(audio))
    augmented_audio = audio + 0.005 * noise
    augmented_file_path = file_path.replace('.wav', '_augmented.wav')
    sf.write(augmented_file_path, augmented_audio, sr)
    return augmented_file_path


# Функция для загрузки аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Функция для загрузки датасета
def load_dataset():
    audio_files = []
    texts = []
    labels = []

    for annotation_file, data_dir in zip(ANNOTATION_FILES, DATA_DIR_FILES):
        annotation_path = os.path.join(ANNOTATION_DIR, annotation_file)
        print(f"Загрузка аннотаций из: {annotation_path}")
        training_annotations = load_annotations(annotation_path)

        for annotation in training_annotations:
            audio_filepath = os.path.join(DATA_DIR, data_dir, annotation['audio_filepath'])
            if os.path.exists(audio_filepath):
                if audio_filepath.endswith('.mp3'):
                    audio_filepath = convert_mp3_to_wav(audio_filepath)

                # Подавляем шум перед аугментацией
                denoised_file = reduce_noise(audio_filepath)

                augmented_file = augment_audio(denoised_file)
                audio_files.append(augmented_file)

                text = annotation['text']
                label = annotation['label']
                texts.append(text)  # добавляем текст
                labels.append(label)  # добавляем метку

            else:
                print(f"Файл {audio_filepath} не найден.")
                continue

    print("Загрузка датасета завершена.")
    return audio_files, texts, labels


# Кодирование текста с помощью BERT
def encode_with_bert(texts, tokenizer, bert_model):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Функция для обучения модели
def train_model(train_loader, model, criterion, optimizer, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        for audio_files, texts, labels in train_loader:
            optimizer.zero_grad()
            text_vectors = encode_with_bert(texts, tokenizer, bert_model)
            outputs = model(text_vectors.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    print("Обучение завершено.")

# Функция для оценки модели
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for audio_files, texts, labels in test_loader:
            text_vectors = encode_with_bert(texts, tokenizer, bert_model)
            outputs = model(text_vectors.float())
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.numpy())
            all_labels.extend(labels.numpy())

    return all_labels, all_preds

# Инициализация BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()

# Основной код для загрузки и обучения модели
audio_files, texts, labels = load_dataset()
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

# Подготовка данных
train_dataset = AudioTextDataset(audio_files, train_texts, train_labels)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

num_classes = len(set(labels))
classifier = TextClassifier(input_dim=768, hidden_dim=128, output_dim=num_classes)  # BERT output is 768
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

# Обучение модели
train_model(train_loader, classifier, criterion, optimizer)

# Оценка модели
test_dataset = AudioTextDataset(audio_files, test_texts, test_labels)
test_loader = DataLoader(test_dataset, batch_size=4)
all_labels, all_preds = evaluate_model(classifier, test_loader)

# Вывод результатов
print(classification_report(all_labels, all_preds, target_names=_labels.values()))


Модель Vosk загружена успешно.
Загрузка аннотаций из: ../data/train/annotation/hr_bot_synt.json
Загрузка датасета завершена.
Обучение завершено.


ValueError: Number of classes, 21, does not match size of target_names, 23. Try specifying the labels parameter